# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [1]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Acquisition et caractéristiques de la lumière

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

Réponse:

1) Capture de l'image brute;
2) Conversion des données analogues en données digitales;
3) Traitement de l'image;
4) Encodage et écriture de l'image.

### Q1.2

Quelle est la différence entre paramètres intrinsèques et extrinsèques du caméra ? Décrivez chaque type en une phrase.

Les paramètres intrinsèques sont les paramètres propres à la caméra, tels que:

- Distance focale;
- Taille du capteur;
- *Shutter speed*.

Les paramètres extrinsèques de la caméra ne sont pas propres à une caméra, et définient son environnement, tels que:

- Repère(s), et matrice(s) de transformation;
- Pose de la caméra.



### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ?

In [2]:
# Réponse ici.
K = np.array(
    [[620.0, 0.0, 1024.0], [0.0, 620.0, 512.0], [0.0, 0.0, 1.0]], dtype=np.float64
)
width_mm, height_mm = (30, 15)
width_px, height_px = (K[0, 2] * 2, K[1, 2] * 2)
f_px = K[0, 0]
f = f_px * width_mm / width_px  # mm

print(f"Distance focale (mm): {f:.3f}")

Distance focale (mm): 9.082


### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement.

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

Réponse:

Caractéristiques intrinsèques:

- Distortion optique (barillet, coussinet);
- Aberrations chromatiques;
- Bruit du capteur;
- Non-uniformité de la sensibilité des pixels;
- Vignetage (*vignetting*);
- Erreurs de calibration (*e.g.* distance focale, centre optique).

Caractéristiques extrinsèques:

- Erreurs dans la pose (position, orientation);
- Vibrations, ou mouvements involontaires;
- Mauvaise estimation des repères;
- Dynamicité de l'environnement (*e.g.* température, lumière);
- Flexion, ou déplacement du support de la caméra.

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

Réponse:

Effectuer une calibration expérimentale:

- Capturer des images d'un motif connu (*e.g.* damier), puis d'utiliser les algorithmes de calibration pour estimer les paramètres intrinsèques et extrinsèques;
- Mesurer les effets de l'environnement, et corriger les erreurs par des modèles ou des ajustements logiciels.

### Q1.5

Dans cette image synthétique : 

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

Réponse:

Les reflets spéculaires correspondent aux points brillants sur les deux boules, et l'illumination diffuse correspond ~~aux parties ombragées de façon *smooth*.~~

Solutionnaire: l'illumination diffuse correspond aux couleurs "retransmises" par les boules (*i.e.* les couleurs qui n'ont pas été absorbées).

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Vous pouvez répondre en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

Réponse:

Les éléments nécessaires sont les angles relatifs à la surface des directions incidentes et réfléchies des sources de lumières, et la longueur d'onde de la lumière.


### Q1.6

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

Réponse:

Des facteurs peuvent influencer la représentation digitale de l'information capturée:

- Technologie des capteurs;
- Disposition des filtres Bayer;
- Paramètres d'équilibrage de la couleur (*white balance*);
- Algorithmes de traitement d'image;
- Portée dynamique, et exposition;
- Caractéristiques des lentilles;
- Calibration et/ou profiles de couleur;
- Conditions de l'éclairage.


**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

Réponse:

Une démarche de traitement d'image (*i.e.* un *pipeline*):

1. Conversion vers un espace de couleurs commun;
2. Algorithme pour quantifier la différence de couleur (*e.g.* distance euclidenne, CIE76, CIE94, CIEDE2000);
3. Normalisation de la portée des valeurs digitales;
4. Analyse statistique pour obtenir la moyenne, la médiane, et l'écart-type des différentes couleurs;
5. Visualisation des couleurs dans un espace de couleurs (*e.g.* graphique 3D RGB, graphique 2D CIE);
6. (±) Définir un seuil de différence de couleurs raisonnable.


## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [3]:
_1_T_0 = np.array([
    [0, 1, 0, 240],
    [1, 0, 0, 80],
    [0, 0, -1, 120],
    [0, 0, 0, 1]
], dtype=np.float64
)
print("0_T_1:\n", _1_T_0)

0_T_1:
 [[  0.   1.   0. 240.]
 [  1.   0.   0.  80.]
 [  0.   0.  -1. 120.]
 [  0.   0.   0.   1.]]


**b)** Trouvez maintenant la transformation inverse.

In [4]:
_1_T_0 = np.array([
    [0, 1, 0, -80],
    [1, 0, 0, -240],
    [0, 0, -1, 120],
    [0, 0, 0, 1],
], dtype=np.float64
)
print("1_T_0:\n", _1_T_0)

1_T_0:
 [[   0.    1.    0.  -80.]
 [   1.    0.    0. -240.]
 [   0.    0.   -1.  120.]
 [   0.    0.    0.    1.]]


**c)** Soit le point $p_0 = [80, 50, 10]^T$, en mm, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [5]:
p_0 = np.array([80, 50, 10, 1], dtype=np.float64)
# p_1 = _1_T_0[:3, :3] @ p_0 + _1_T_0[:3, 3]  # inhomogeneous coordinates
p_1 = _1_T_0 @ p_0  # homogeneous coordinates
print("p_1:\n", p_1)

p_1:
 [ -30. -160.  110.    1.]


### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$.

In [ ]:
print("K:\n", K)
_1_P_0 = K @ _1_T_0[:3, :]

_1_P_0_tilde = np.array(
    [
        [_1_P_0[0, 0], _1_P_0[0, 1], _1_P_0[0, 2], _1_P_0[0, 3]],
        [_1_P_0[1, 0], _1_P_0[1, 1], _1_P_0[1, 2], _1_P_0[1, 3]],
        [_1_P_0[2, 0], _1_P_0[2, 1], _1_P_0[2, 2], _1_P_0[2, 3]],
        [0, 0, 0, 1],
    ],
    dtype=np.float64,
)

print(f"1_P_0_tilde:\n{_1_P_0_tilde}")

K:
 [[6.200e+02 0.000e+00 1.024e+03]
 [0.000e+00 6.200e+02 5.120e+02]
 [0.000e+00 0.000e+00 1.000e+00]]
0_P_1:
[[ 0.000e+00  6.200e+02 -1.024e+03  7.328e+04]
 [ 6.200e+02  0.000e+00 -5.120e+02 -8.736e+04]
 [ 0.000e+00  0.000e+00 -1.000e+00  1.200e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


**b)** Soit le point $p_0 = [250, 10, 0]$, en mm. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra.

In [7]:
p_0 = np.array([250, 10, 0, 1], np.float64)

X_s_prime = _1_P_0_tilde @ p_0
X_s = X_s_prime / X_s_prime[2]

print("X_s:\n", X_s)

X_s:
 [6.62333333e+02 5.63666667e+02 1.00000000e+00 8.33333333e-03]


## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [8]:
p_max = np.array([0, 1024, 1, 1 / 120], dtype=np.float64)

# (W_px/2) / Z_c = W_mm / f_px ???

### Q3.2

Soit le point $x_s = [120, 200]$, en pixels, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [9]:
x_s = np.array([120,200])